In [3]:
import ssl
ssl._create_default_https_context=ssl._create_unverified_context

In [4]:
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# nomalize
x_train, x_test = x_train / 255, x_test / 255
y_train_ohe = to_categorical(y_train, num_classes=10)
y_test_ohe = to_categorical(y_test, num_classes=10)

# type(x_train) # numpy.ndarray,
# x_train.shape # (50000, 32, 32, 3)

170498071/170498071 [==============================] - 54s 0us/step


In [5]:
alice_x_train=x_train[:, :16]
alice_x_test=x_test[:,:16]
bob_x_train=x_train[:,16:]
bob_x_test=x_test[:,16:]

In [6]:
print(f"alice train dataset size:{alice_x_train.shape}, \n\
bob train dataset size: {bob_x_train.shape},\n\
alice test dataset size: {alice_x_test.shape},\n\
label train dataset size: {y_train_ohe.shape}")


alice train dataset size:(50000, 16, 32, 3), 
bob train dataset size: (50000, 16, 32, 3),
alice test dataset size: (10000, 16, 32, 3),
label train dataset size: (50000, 10)


In [7]:
import numpy as np
from secretflow.data.ndarray import load
import secretflow as sf
from secretflow.ml.nn import SLModel

sf.shutdown( )
sf.init(['alice','bob'],address='local')
alice,bob=sf.PYU('alice'),sf.PYU('bob')

np.savez(
    "alice.npz",
    train_x=alice_x_train,
    train_y=y_train_ohe,
    test_x=alice_x_test,
    test_y=y_test_ohe
)
np.savez(
    "bob.npz",
    train_x=bob_x_train,
    train_y=None,
    test_x=bob_x_test,
    test_y=None
)
fed_npz = load(
    {alice: "./alice.npz", bob: "./bob.npz"},
    partition_way=sf.data.ndarray.PartitionWay.VERTICAL,
    allow_pickle=True
)
fed_train_x, fed_train_y, fed_test_x, fed_test_y = (
    fed_npz["train_x"],
    fed_npz["train_y"],
    fed_npz["test_x"],
    fed_npz["test_y"],
)
fed_npz


2024-08-06 14:37:03,263	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=3.92gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-08-06 14:37:03,386	INFO worker.py:1724 -- Started a local Ray instance.
(raylet) Spilled 4696 MiB, 8 objects, write throughput 1054 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 6339 MiB, 12 objects, write throughput 1074 MiB/s.


{'train_x': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f8849532440>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f8a32a966b0>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>),
 'train_y': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f8849704880>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f8849e7d2d0>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>),
 'test_x': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f8849e767d0>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f8a4b06d420>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>),
 'test_y': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f8849e76080>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f8849530040>}, partition_way=<Partiti

# 明文模型

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import train_test_split

class SplitMLP(tf.keras.Model):
    def __init__ (self):
        super(SplitMLP,self).__init__()
        self.alice_model= keras.Sequential(
            [
                layers.Flatten(),
                layers.Dense(100,activation="relu"),
                layers.Dense(32,activation="relu")
            ]
        )
        self.bob_model=keras.Sequential(
            [
                layers.Flatten(),
                layers.Dense(100, activation="relu"),
                layers.Dense(32,activation="relu")
            ]
        )
        self.fuse_layer=layers.Dense(64,activation='relu')
        self.output_layer= layers.Dense(10,activation='softmax')
    def __call__(self,inputs):
        alice_feature = inputs['alice_feature']
        bob_feature = inputs['bob_feature']
        # hiddens
        h_alice=self.alice_model(alice_feature)
        h_bob= self.bob_model(bob_feature)
        merged_layer=layers.concatenate([h_alice,h_bob])
        fuse_layer_out=self.fuse_layer(merged_layer)
        output=self.output_layer(fuse_layer_out)
        return output

In [9]:
alice_input= tf.keras.Input(shape=(16,32,3),name='alice_feature')
bob_input =tf.keras.Input(shape=(16,32,3),name='bob_feature')
split_mlp=SplitMLP()
logits= split_mlp({'alice_feature': alice_input, 'bob_feature': bob_input})
model= tf.keras.Model(
    inputs=[alice_input, bob_input],
    outputs=logits
)
model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=[tf.keras.metrics.AUC()]
)
train_dataset=tf.data.Dataset.from_tensor_slices(
    (
        {
            "alice_feature": alice_x_train,
            "bob_feature": bob_x_train
        },
        y_train_ohe,
    )
).batch(32)
model.fit(train_dataset,epochs=3)

Epoch 1/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2806 - auc: 0.7805
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2511 - auc: 0.8365
Epoch 3/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2407 - auc: 0.8526


# 密文模型

## 1. 基础模型创建

In [10]:
def create_base_model(input_dim, output_dim, name='base_model'):
    #Create model
    def create_model():
        from tensorflow import keras
        from tensorflow.keras import layers
        import tensorflow as tf
        # base model config
        model = keras.Sequential(
            [
                keras.Input(shape=input_dim),
                layers.Flatten(),
                layers.Dense(100,activation="relu"),
                layers.Dense(output_dim, activation="relu")
            ]
        )
        # Compile model
        model.summary()
        model.compile(
            # 模型分布在各方，需要compile才能前向后向传播
            loss='binary_crossentropy',
            optimizer='adam',
            metrics=["accuracy",tf.keras.metrics.AUC()]
        )
        return model
    return create_model

In [11]:
# prepare model
hidden_size = 32
model_base_alice=create_base_model((16, 32, 3),hidden_size)
model_base_bob =create_base_model((16, 32, 3),hidden_size)
model_base_alice()
model_base_bob()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 1536)              0         
                                                                 
 dense_6 (Dense)             (None, 100)               153700    
                                                                 
 dense_7 (Dense)             (None, 32)                3232      
                                                                 
Total params: 156,932
Trainable params: 156,932
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 1536)              0         
                                                                 
 dense_8 (Dense)          

In [12]:
def create_fuse_model(input_dim, output_dim, party_nums, name='fuse_model'):
    def create_model():
        from tensorflow import keras
        from tensorflow.keras import layers
        import tensorflow as tf
        # input
        input_layers =[]
        for i in range(party_nums):
            input_layers.append(
                keras.Input(
                    input_dim,
                )
            )
        merged_layer= layers.concatenate(input_layers)
        fuse_layer = layers.Dense(64,activation='relu')(merged_layer)
        output =layers.Dense(output_dim,activation='softmax')(fuse_layer)
        model = keras.Model(inputs=input_layers, outputs=output)
        model.summary()
        model.compile(
            loss='binary_crossentropy',
            optimizer='adam',
            metrics=[tf.keras.metrics.AUC()]
        )
        return model

    return create_model

In [13]:
model_fuse=create_fuse_model(input_dim=hidden_size, party_nums=2,output_dim=10)
model_fuse()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 64)           0           ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense_10 (Dense)               (None, 64)           4160        ['concatenate_1[0][0]']    

## 2. 拆分模型创建

In [14]:
base_model_dict ={alice: model_base_alice, bob: model_base_bob}


In [15]:
from secretflow.security.privacy import DPStrategy, LabelDP
from secretflow.security.privacy.mechanism.tensorflow import GaussianEmbeddingDP

# Define DP operations
train_batch_size = 128
gaussian_embedding_dp = GaussianEmbeddingDP(
    noise_multiplier=0.5,
    l2_norm_clip=1.0,
    batch_size=train_batch_size,
    num_samples=1000,
    is_secure_generator=False,
)
label_dp = LabelDP(eps=64.0)
dp_strategy_alice = DPStrategy(label_dp=label_dp)
dp_strategy_bob = DPStrategy(embedding_dp=gaussian_embedding_dp)
dp_strategy_dict = {alice: dp_strategy_alice, bob: dp_strategy_bob}
dp_spent_step_freq = 10

In [16]:
sl_model = SLModel(
    base_model_dict=base_model_dict,
    device_y=alice,
    model_fuse=model_fuse,
    dp_strategy_dict=dp_strategy_dict
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.tensorflow.sl_base.PYUSLTFModel'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.tensorflow.sl_base.PYUSLTFModel'> with party bob.


In [17]:
history = sl_model.fit(
    fed_train_x,
    fed_train_y,
    validation_data=(fed_test_x, fed_test_y),
    epochs=3,
    batch_size=train_batch_size,
    shuffle=True,
    verbose=1,
    validation_freq=1,
    dp_spent_step_freq=dp_spent_step_freq,
)

INFO:root:SL Train Params: {'self': <secretflow.ml.nn.sl.sl_model.SLModel object at 0x7f88a456dae0>, 'x': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f8849532440>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f8a32a966b0>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>), 'y': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f8849704880>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f8849e7d2d0>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>), 'batch_size': 128, 'epochs': 3, 'verbose': 1, 'callbacks': None, 'validation_data': (FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f8849e767d0>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f8a4b06d420>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>), FedNdarray(partitions={PYURuntime(alice): <secretflow.device.devic

Epoch 1/3


Train Processing: :   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 2/3


Train Processing: :   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 3/3


Train Processing: : 100%|█████████▉| 390/391 [00:16<00:00, 23.01it/s, {'train_loss': 0.27049786, 'train_auc_1': 0.8010061, 'val_loss': 0.2664643, 'val_auc_1': 0.80855715}]


In [18]:
global_metric = sl_model.evaluate(fed_test_x, fed_test_y, batch_size=128)

Evaluate Processing: :  99%|█████████▊| 78/79 [00:00<00:00, 96.49it/s, loss=0.267, auc_1=0.806]
